# auto-img-tagによる学習データの自動生成と、YOLOv5による学習と推論

テストデータのジャンル( **genre** )として、バナナ(banana)と野菜(vegetable)とお菓子(candy)を用意しています。
<br/>

下のコードセルで実行時のオプションを選択してください。
<br/>
<br/>

* bananaはクラス(カテゴリー)が１個だけでデータ数が少ないので、最初はbananaを試してみてください。
<br/>

* vegetableのクラスの数は６個で、candyのクラスの数は１０個です。

    これらはデータ数が多く、YOLOv5でキャッシュを使うとメモリ不足になるので
    
     **Google Colab Pro** の **ハイメモリ** でない場合は **use_cache** をオフにしてください。

     Colab Proは有料ですが月1000円のお手頃価格なので個人的にはオススメです。
<br/>

* vegetableでは質感が重要なので画像サイズ( **img_size** )は480よりも640の方が成績が良いようです。

    candyは480でも、そこそこ検出できています。
<br/>

*  **data_size** は1クラスあたりの学習データの数です。

    この値を大きくすると学習時間はかかりますが、精度は良くなります。
<br/>

学習時間の目安は以下を参考にしてください。

| genre     | use_cache | img_size | data_size | 学習時間 |
| :----     | :------:  | :------: | :-------: |----------|
| banana    | ON        | 640      | 2000      | 約1時間 |
| vegetable | ON        | 640      | 1000      | 約3時間 |
| candy     | ON        | 480      | 1000      | 約3時間 |
<br/>
<br/>

In [ ]:
genre = 'banana' #@param ["banana", "vegetable", "candy"]

use_cache = True #@param {type:"boolean"}

img_size = "640" #@param ['640', '480']

data_size = 1000 #@param {type:"integer"}

### パッケージをアップデートする。

In [ ]:
# YOLOv5の実行時にalbumentationsのバージョンが古いという警告が出たのでアップデートする。
# albumentationsだけアップデートするとOpenCVがエラーになったので一緒にアップデートする。
%pip install -U opencv-python albumentations

# 学習データ作成用の動画ファイルをダウンロードする。

In [ ]:
! wget https://uroa.jp/auto-img-tag/data/$genre-video.zip

# ダウンロードしたファイルを解凍する。
! unzip -q $genre-video.zip

# テスト用の画像ファイルをダウンロードする。

In [ ]:
! wget https://uroa.jp/auto-img-tag/data/$genre-test.zip

# ダウンロードしたファイルを解凍する。
! unzip -q $genre-test.zip

# COCO データセットをダウンロードする。

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

# ダウンロードしたファイルを解凍する。
!unzip -q val2017.zip

# auto-img-tagをクローンする。

In [ ]:
!git clone https://github.com/teatime77/auto-img-tag.git

# 動画ファイルからCOCO形式の学習データを作成する。

In [ ]:
%cd /content/auto-img-tag
! time python main.py -i /content/$genre-video -bg /content/val2017 -o /content -dtsz $data_size -imsz $img_size
%cd /content

# COCO形式からYOLOv5形式に学習データを変換する。

In [ ]:
! python auto-img-tag/yolov5.py

# YOLOv5の設定ファイルを表示する。
%cat /content/datasets/data.yaml

## 偽陽性対策として背景画像ファイルをトレーニング画像に追加する。

In [ ]:
import glob
import random
import shutil

# 背景画像ファイルのリスト
bg_files = list(glob.glob('/content/val2017/*.jpg'))

# 背景画像50個をトレーニング画像に追加する。
for idx, bg_file in enumerate(random.sample(bg_files, 50)):
    shutil.copyfile(bg_file, f'/content/datasets/images/train/bg_file-{idx}.jpg')

# YOLOv5をインストールする。

In [ ]:
%cd /content

#YOLOv5をcloneする。
!git clone https://github.com/ultralytics/yolov5 

%cd yolov5

# 必要なパッケージをインストールする。
%pip install -qr requirements.txt

import torch
import os
from IPython.display import Image, clear_output  # to display images

# PyTorchのバージョンとGPUの種類を表示する。
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# 学習を開始する。

In [ ]:
# 開始時刻を表示する。
! TZ=Asia/Tokyo date

# キャッシュの指定
if use_cache:
    cache = '--cache'
else:
    cache = ''

# 学習を開始する。
! python train.py --batch-size -1 --epochs 150 --data /content/datasets/data.yaml --weights yolov5s.pt --patience 50 --exist-ok --img $img_size $cache

# TensorBoardに学習の経過を表示する。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

# テスト用の画像ファイルで推論をする。

In [ ]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.1 --source /content/$genre-test

# 推論の結果を表示する。

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'):
    display(Image(filename=imageName))

# 学習と推論の結果をダウンロードする。

In [ ]:
from google.colab import files

# 学習と推論の結果をZIPで圧縮する。
! zip -q -r $genre-result runs

# ZIPファイルをダウンロードする。
files.download(f'{genre}-result.zip')